In [11]:
from model import TextRNN
from utils import *
from model import *
import sys
import torch.optim as optim
from torch import nn
import torch
from sklearn.metrics import accuracy_score, classification_report

In [2]:
train_file = '/Volumes/data/Data Science/Diploma/Detect-emotion-sentimental/dataset/booking/booking-sentences-train.csv'
test_file = '/Volumes/data/Data Science/Diploma/Detect-emotion-sentimental/dataset/booking/booking-sentences-test.csv'
val_file = '/Volumes/data/Data Science/Diploma/Detect-emotion-sentimental/dataset/booking/booking-sentences-val.csv'
w2v_file = '../data/ubercorpus.cased.tokenized.word2vec.300d'

In [23]:
class Config(object):
    embed_size = 300
    hidden_layers = 2
    hidden_size = 32
    bidirectional = True
    output_size = 2
    max_epochs = 10
    lr = 0.25
    batch_size = 64
    max_sen_len = 20 # Sequence length for RNN
    dropout_keep = 0.8

In [4]:
config = Config()

In [24]:
dataset = Dataset(config)
dataset.load_data(w2v_file, train_file, test_file)

KeyboardInterrupt: 

 Create Model with specified optimizer and loss function

In [ ]:
model = TextRNN(config, len(dataset.vocab), dataset.word_embeddings)
if torch.cuda.is_available():
    model.cuda()
model.train()
optimizer = optim.SGD(model.parameters(), lr=config.lr)
NLLLoss = nn.NLLLoss()
model.add_optimizer(optimizer)
model.add_loss_op(NLLLoss)

In [ ]:
train_losses = []
val_accuracies = []

In [ ]:
def train():
    for i in range(config.max_epochs):
        print ("Epoch: {}".format(i))
        train_loss,val_accuracy = model.run_epoch(dataset.train_iterator, dataset.val_iterator, i)
        train_losses.append(train_loss)
        val_accuracies.append(val_accuracy)


In [ ]:
%time train()

In [16]:
def calc_true_and_pred(model, iterator):
    all_preds = []
    all_y = []
    for idx,batch in enumerate(iterator):
        if torch.cuda.is_available():
            x = batch.text.cuda()
        else:
            x = batch.text
        y_pred = model(x)
        predicted = torch.max(y_pred.cpu().data, 1)[1] + 1
        all_preds.extend(predicted.numpy())
        all_y.extend(batch.label.numpy())
    return all_y, all_preds

In [17]:
all_y, all_preds = calc_true_and_pred(model, dataset.test_iterator)

### accuracy score

In [19]:
test_acc = accuracy_score(all_y, np.array(all_preds).flatten())
print ('Final Test Accuracy: {:.4f}'.format(test_acc))

Final Test Accuracy: 0.3099


### confusion matrix

In [20]:
report = classification_report(all_y, np.array(all_preds).flatten())

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [22]:
print(report)

              precision    recall  f1-score   support

           1       0.48      0.23      0.31     56013
           2       0.52      0.38      0.44     63836
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0

   micro avg       0.31      0.31      0.31    119849
   macro avg       0.25      0.15      0.19    119849
weighted avg       0.50      0.31      0.38    119849

